In [ ]:
import cv2
import glob
import numpy as np
from sklearn.model_selection import train_test_split

In [ ]:
vehicles = glob.glob("./Vehicles/*.png")

In [ ]:
non_vehicles = glob.glob("./Non_Vehicles/*.png")

In [ ]:
Y = np.concatenate([np.array([[1,0]]*len(vehicles)), 
                    np.array([[0,1]]*len(non_vehicles))])

In [ ]:
x = []
for name in vehicles:    
    x.append(cv2.cvtColor(cv2.imread(name), cv2.COLOR_BGR2RGB))
for name in non_vehicles:    
    x.append(cv2.cvtColor(cv2.imread(name), cv2.COLOR_BGR2RGB))


In [ ]:

print(x_train.shape)
print(x_train.shape[0])
print(x_test.shape[0])
input_shape =  (3,64,64)

In [ ]:
import tensorflow as tf

In [ ]:
with tf.Session() as sess:
    tf_data = tf.placeholder(tf.float32, shape=(None, None, None, 3), name="image_input")
    tf_labels = tf.placeholder(tf.float32, shape=(None),name="labels_input")
    tf_train = tf.placeholder(tf.bool,name="train_input")
    batches = (int(np.ceil(x_train.shape[0]/64.)))
    norm = tf_data/127.5 - 1
    c1 = tf.layers.conv2d(norm, 16, 3, strides=(1, 1), padding='SAME',
                            kernel_initializer=tf.truncated_normal_initializer(stddev=0.001), activation=tf.nn.relu)
    c2 = tf.layers.conv2d(c1, 16, 3, strides=(1, 1), padding='SAME',
                            kernel_initializer=tf.truncated_normal_initializer(stddev=0.001), activation=tf.nn.relu)
    p = tf.layers.max_pooling2d(c2, 8, strides=(8,8), padding='SAME')
    
    drop1 = tf.layers.dropout(p,rate=0.25,training=tf_train)
    
    c3 = tf.layers.conv2d(drop1, 128, 8, strides=(1, 1), padding='VALID',
        kernel_initializer=tf.truncated_normal_initializer(stddev=0.001), activation=tf.nn.relu)
    
    drop2 = tf.layers.dropout(c3,rate=0.50,training=tf_train)
    c4 = tf.layers.conv2d(drop2, 2, 1, strides=(1, 1), padding='VALID',
        kernel_initializer=tf.truncated_normal_initializer(stddev=0.001))
    sm = tf.nn.softmax(c4, name="output")
    
    logits = tf.reshape(c4,(-1,2),name='logits')
    labels = tf.reshape(sm,(-1,2),name='lables')
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=labels))
    train_op = tf.train.AdamOptimizer(0.001).minimize(loss)
    
    correct_prediction = tf.equal(tf.argmax(logits,1), tf.argmax(labels,1))
    tf_accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    
    for epoch in range(20):
        for batch in range(batches):
            train_loss, _, accuracy = sess.run([loss, train_op, tf_accuracy],
                        feed_dict={tf_data: x_train[batch*64:batch*64+64], 
                                   tf_labels: Y_train[batch*64:batch*64+64], 
                                   tf_train: True})
        print( accuracy)
        
        train_loss, accuracy = sess.run([l, tf_accuracy],feed_dict={tf_data: x_test, tf_labels: Y_test, 
                                   tf_train: False})
        print( accuracy)
        